# Overcoming Echo Chambers in Recommendation Systems

## Introduction



Continue reading to learn more about how this project was developed or skip to the <a href = #Get_Recommendations> Get Recommendations</a> in Section 2 to test the recommendation engine.

# Recommendations

## Local Code Imports

In [1]:
# DO NOT REMOVE THESE
%load_ext autoreload
%autoreload 2

In [2]:
# DO NOT REMOVE This
%reload_ext autoreload

In [3]:
# Uncomment to filter warnings
import warnings
warnings.filterwarnings('ignore')

In [9]:
# DO NOT REMOVE
# import local src module -
from src import recapp as ra

<a id='Get_Recommendations'></a>

## Get Recommendations

In [12]:
ra.get_recommendations()

Enter a ranking from 1 (lowest) to 5 (highest) for the following movies.
If you have not seen the movie, press enter.
Human Planet (2011): 4
Rear Window (1954): 
Last Year's Snow Was Falling (1983): 6
Ratings must be whole numbers between 1 and 5.
City of God (Cidade de Deus) (2002): 3
American Beauty (1999): 3
Grand Illusion (La grande illusion) (1937): 3
Sherlock - A Study in Pink (2010): 3
Vertigo (1958): 4
The Adventures of Sherlock Holmes and Dr. Watson: Bloody Signature (1979): 42
Ratings must be whole numbers between 1 and 5.
Maltese Falcon, The (1941): 4
Rashomon (Rashômon) (1950): 2
Godfather, The (1972): 4
Usual Suspects, The (1995): 2




Try:
    I, Claudius (1976)
    Planet Earth (2006)
    Planet Earth II (2016)
    Dylan Moran: Yeah, Yeah (2011)
    Harakiri (Seppuku) (1962)
    Human Condition III, The (Ningen no joken III) (1961)
    Civil War, The (1990)
    Smiley's People (1982)
    Life (2009)
    Blue Planet II (2017)


You may also like:
    Quiz Show (1994)
   